<a href="https://colab.research.google.com/github/JonatanTur/Vic3-loop-dependency-factory/blob/main/Multifactory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from math import ceil

In [2]:
class Building:
  def __init__(self, name, inputs:dict, outputs:dict, workers):
    self.name = name
    self.inputs = inputs
    self.outputs = outputs
    self.workers = workers

In [3]:
buildings = []
to_build = {}

In [4]:
#Buildings
Steelmill = Building(name="Steelmill",inputs={"iron":50, "coal":10}, outputs={"steel":80}, workers=5000)
buildings.append(Steelmill)

Mine = Building("Mine",{"steel":10}, {"iron":25, "coal":5}, 1000)
buildings.append(Mine)

In [6]:
needed = {}
surplus = {}
workers = 0

In [7]:
print(type(needed),
type(surplus),
type(to_build))

<class 'dict'> <class 'dict'> <class 'dict'>


In [8]:
def construct(output):
  global needed, surplus, buildings, to_build

  for resource, amount in output.items():
    needed[resource] = amount
  while any(amount > 0 for amount in needed.values()):
    for resource in list(needed.keys()):
      if resource in list(surplus.keys()) and surplus[resource] > 0:
        if surplus[resource] >= needed[resource]:
          surplus[resource] -= needed[resource]
          needed[resource] = 0
        else:
          needed[resource] -= surplus[resource]
          surplus[resource] = 0

      if needed[resource] > 0:
        for bld in buildings:
          if resource in bld.outputs:
            needed_quantity = needed[resource]
            singular_bld_rsc_quantity = bld.outputs[resource]
            blds_to_build = ceil(needed_quantity/ singular_bld_rsc_quantity)

            to_build[bld.name] = to_build.get(bld.name, 0) + blds_to_build
            workers += bld.workers * blds_to_build


            for res, quan in bld.outputs.items():
               produced = quan * blds_to_build

               if res in needed:
                 if produced >= needed[res]:
                   surplus[res] = surplus.get(res, 0) + (produced - needed[res])
                   needed[res] = 0
                 else:
                   needed[res] -= produced
               else:
                   surplus[res] = surplus.get(res, 0) + produced

            for res, quan in bld.inputs.items():
               needed[res] = needed.get(res, 0) + quan * blds_to_build
            print('we need: ',needed,"\n we have too much:",surplus)
            break

In [9]:
construct({"steel": 800})
print(to_build, "\n\n", needed, surplus)

we need:  {'steel': 0, 'iron': 500, 'coal': 100} 
 we have too much: {'steel': 0}
we need:  {'steel': 200, 'iron': 0, 'coal': 0} 
 we have too much: {'steel': 0, 'iron': 0, 'coal': 0}
we need:  {'steel': 0, 'iron': 150, 'coal': 30} 
 we have too much: {'steel': 40, 'iron': 0, 'coal': 0}
we need:  {'steel': 60, 'iron': 0, 'coal': 0} 
 we have too much: {'steel': 40, 'iron': 0, 'coal': 0}
we need:  {'steel': 0, 'iron': 50, 'coal': 10} 
 we have too much: {'steel': 60, 'iron': 0, 'coal': 0}
we need:  {'steel': 20, 'iron': 0, 'coal': 0} 
 we have too much: {'steel': 60, 'iron': 0, 'coal': 0}
{'Steelmill': 14, 'Mine': 28} 

 {'steel': 0, 'iron': 0, 'coal': 0} {'steel': 40, 'iron': 0, 'coal': 0}
